# Take 2: Adaboost with Weak Classifiers
None of that bagging classifier bullshit

In [19]:
# Import guys
import pandas as pd
import numpy as np
from scipy import sparse
# from AdaBoostClassifier import AdaBoostClassifier
#%run AdaBoostWeak.py
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


### Import data

In [20]:
# column names
with open('data/column_names.txt', 'r') as f:
    column_names = [line.strip() for line in f]

sparse_dat = sparse.load_npz("data/sparse_df.npz")

# Extract labels from the first column
labels = sparse_dat[:, 0]

# Create a list of column indices to keep
to_keep = list(set(range(sparse_dat.shape[1])) - set([0]))

# Extract the design matrix
X = sparse_dat[:, to_keep]

print(f'labels: \n{labels}')

labels: 
  (2, 0)	1
  (5, 0)	1
  (8, 0)	1
  (9, 0)	1
  (11, 0)	1
  (12, 0)	1
  (15, 0)	1
  (19, 0)	1
  (34, 0)	1
  (42, 0)	1
  (54, 0)	1
  (56, 0)	1
  (65, 0)	1
  (67, 0)	1
  (68, 0)	1
  (93, 0)	1
  (95, 0)	1
  (114, 0)	1
  (117, 0)	1
  (120, 0)	1
  (121, 0)	1
  (123, 0)	1
  (134, 0)	1
  (135, 0)	1
  (139, 0)	1
  :	:
  (15952, 0)	1
  (15953, 0)	1
  (15954, 0)	1
  (15955, 0)	1
  (15956, 0)	1
  (15957, 0)	1
  (15958, 0)	1
  (15959, 0)	1
  (15960, 0)	1
  (15961, 0)	1
  (15962, 0)	1
  (15963, 0)	1
  (15964, 0)	1
  (15965, 0)	1
  (15966, 0)	1
  (15967, 0)	1
  (15968, 0)	1
  (15969, 0)	1
  (15970, 0)	1
  (15971, 0)	1
  (15972, 0)	1
  (15973, 0)	1
  (15974, 0)	1
  (15975, 0)	1
  (15976, 0)	1


In [21]:
print(f'Design matrix: \n{X}')

Design matrix: 
  (17173, 0)	1
  (17240, 0)	2
  (17164, 1)	1
  (17363, 1)	1
  (17448, 1)	1
  (17910, 1)	1
  (17914, 1)	1
  (17933, 1)	2
  (15801, 2)	1
  (15867, 2)	1
  (16217, 2)	1
  (17173, 2)	3
  (17189, 2)	1
  (17386, 2)	1
  (17765, 2)	4
  (17933, 3)	2
  (17325, 4)	1
  (17366, 4)	1
  (16271, 5)	1
  (16837, 5)	1
  (17933, 6)	2
  (17933, 7)	2
  (17173, 8)	2
  (17933, 8)	1
  (15801, 9)	1
  :	:
  (8386, 56199)	2
  (8669, 56200)	1
  (10316, 56200)	1
  (8805, 56201)	1
  (9218, 56201)	1
  (9565, 56202)	1
  (9814, 56202)	1
  (6232, 56203)	1
  (9353, 56203)	1
  (5661, 56204)	9
  (6860, 56204)	1
  (8698, 56204)	1
  (125, 56205)	1
  (1228, 56205)	1
  (4422, 56205)	1
  (5648, 56206)	1
  (7464, 56206)	1
  (5648, 56207)	2
  (6025, 56208)	2
  (6025, 56209)	2
  (6151, 56210)	2
  (8669, 56211)	1
  (10316, 56211)	1
  (6340, 56212)	1
  (8696, 56212)	1


### Train and Test split
To do: consider stratifying by type

In [22]:
# To do - stratify the split 
n_samples = labels.shape[0]
# Use train_test_split.
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, shuffle=False)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(y_train.dtype)
print("proportion of spam in training data:", (y_train == 1).sum().item() / y_train.shape[0])
print("proportion of spam in testing data:", (y_test == 1).sum().item() / y_test.shape[0])

(14524, 56213)
(14524, 1)
(3632, 56213)
(3632, 1)
int64
proportion of spam in training data: 0.39568989259157256
proportion of spam in testing data: 0.11921806167400881


In [5]:
def errors(y, y_pred):
        """
        Calculate the proportion of type 2 errors - when the true label is 1 - spam, and the predicted label is 0 - ham

        Args:
        y: true labels
        y_pred: predicted labels
        """
        n = y.shape[0]
        type2errors = ((y == 1) & (y_pred == 0)).sum().item()
        type1errors = ((y == 0) & (y_pred == 1)).sum().item()
        correct = (y_pred == y).sum().item()
        return type2errors, type1errors, correct

### Train/Test models

2 models:
1. Without penalty
2. With penalty

In [25]:
%run AdaBoostWeak.py

aboost1 = AdaBoostWeak(type2penalty = False, rounds = 200, maxDTdepth = 5)
aboost1.fit(X = X_train, y = y_train.toarray().ravel())



In [23]:
predictions = aboost1.predict(X_test)
type2, type1, correct = errors(y_test.toarray().ravel(), predictions)
print("Model 1 Test Accuracy: ", correct/len(predictions))
print(f'unique predictions - should be 0 and 1: {np.unique(predictions)}')
print(f'type 2 errors: {type2} \n type 1 errors: {type1}')
print(aboost1.alphas)
print(predictions)

Model 1 Test Accuracy:  0.6310572687224669
unique predictions - should be 0 and 1: [0 1]
type 2 errors: 11 
 type 1 errors: 1329
[1.9311982474293699, 1.1922207924007637, 0.6385318941643855, 0.9564294022214502, 0.816069472339187, 0.6922920016770775, 0.6369655768031284, 0.5580145171983981, 0.53676580386847, 0.4237535232743738, 0.4623203222935166, 0.4723842325214209, 0.4895676529238302, 0.3666700342477265, 0.35037721839934366, 0.5165341888678217, 0.5573056346856897, 0.4064054111284078, 0.34746557733007816, 0.3810543285918116, 0.2819655300587526, 0.3205254061893938, 0.29391409928509016, 0.3001255376174512, 0.36456174095584926, 0.4132540473505003, 0.32304407528099016, 0.2772103709319034, 0.29892879797327976, 0.29654041317661484, 0.26341418044268283, 0.2785641528371635, 0.2755434935050669, 0.3646419532443164, 0.2985319345378505, 0.20131180360268183, 0.25099494349663737, 0.2451376577056051, 0.2040510624113616, 0.21107882435340944, 0.2061592115648806, 0.20742788270173232, 0.18316633412256617, 

In [13]:
predictions = aboost1.predict(X_train)
type2, type1, correct = errors(y_train.toarray().ravel(), predictions)
print("Model 1 (no penalty) Training set Accuracy: ", correct/len(predictions))
print(f'unique predictions - should be 0 and 1: {np.unique(predictions)}')
print(f'type 2 errors: {type2} \n type 1 errors: {type1}')

Model 1 (no penalty) Training set Accuracy:  0.98347562654916
unique predictions - should be 0 and 1: [0 1]
type 2 errors: 129 
 type 1 errors: 111


### Model tuning/Hyperparameter search

Sigh. So much overfitting

In [26]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score


clf = DecisionTreeClassifier(max_depth=9)
cvs = cross_val_score(clf, X_train,predictions,cv=4)
print("cross val scores ",cvs)


ValueError: Found input variables with inconsistent numbers of samples: [14524, 3632]

In [15]:
# #checkity check
# from importlib import reload
# import AdaBoostWeak
# reload(AdaBoostWeak)
# from AdaBoostWeak import AdaBoostWeak
# a= AdaBoostWeak(rounds=100)
# a.get_params().keys()

In [16]:
# #clonity checkity check
# from sklearn.base import clone
# ada = AdaBoostWeak(rounds=100, type2penalty=False, maxDTdepth=1)  # Replace with actual parameters
# ada_clone = clone(ada)

In [31]:
# #not adaboost classifier woops
# from sklearn.model_selection import GridSearchCV

# clf = DecisionTreeClassifier()
# param_grid = [
#     {'type2penalty': [False] ,'rounds': [200, 300, 400, 500], 'maxDTdepth': [5,6,7,8,9,10]},
#     {'type2penalty': [True] ,'rounds': [200, 300, 400, 500], 'maxDTdepth': [5,6,7,8,9,10]}
#     ]

# print(type(y_train))
# print(y_train.shape)
# y_train_dense = y_train.toarray().ravel()
# print(y_train.shape)

# grid_search = GridSearchCV(clf, param_grid, cv=4)
# grid_search.fit(X_train, y_train_dense)

# print(grid_search.best_params_)


aboost = AdaBoostWeak(rounds=100)
param_grid = [
    {'type2penalty': [False], 'rounds': [200], 'maxDTdepth': [5, 10]},
    {'type2penalty': [True], 'rounds': [200], 'maxDTdepth': [5, 10]}
]

print(type(y_train))
print(y_train.shape)

y_train_dense = y_train.toarray().ravel()

print(y_train_dense.shape)
grid_search = GridSearchCV(aboost, param_grid, cv=4, n_jobs=-1)
grid_search.fit(X_train, y_train_dense)

print("best params:", grid_search.best_params_)


<class 'scipy.sparse.csr.csr_matrix'>
(14524, 1)
(14524,)
best params: {'maxDTdepth': 10, 'rounds': 200, 'type2penalty': True}


In [30]:
# y_test_pred = aboost.predict(X_test)
# test_accuracy = accuracy_score(y_test, y_test_pred)

IndexError: list index out of range

shit takes fucking forever

In [ ]:

aboost = AdaBoostWeakClassic(rounds=100)
param_grid = [
    {'type2penalty': [False], 'rounds': [200, 300, 400, 500], 'maxDTdepth': [5, 6, 7, 8, 9, 10]},
    {'type2penalty': [True], 'rounds': [200, 300, 400, 500], 'maxDTdepth': [5, 6, 7, 8, 9, 10]}
]

print(type(y_train))
print(y_train.shape)

y_train_dense = y_train.toarray().ravel()

print(y_train_dense.shape)
grid_search = GridSearchCV(aboost, param_grid, cv=4)
grid_search.fit(X_train, y_train_dense)

print("best params:", grid_search.best_params_)


In [18]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score
param_grid = [
    {'type2penalty': [False] ,'rounds': [200, 300, 400, 500], 'maxDTdepth': [5,6,7,8,9,10]},
    {'type2penalty': [True] ,'rounds': [200, 300, 400, 500], 'maxDTdepth': [5,6,7,8,9,10]}
    ]


model = AdaBoostWeak(type2penalty=False, maxDTdepth=5,rounds=100)
scorer = make_scorer(accuracy_score)
grid_search=GridSearchCV(estimator=model, param_grid=param_grid, scoring=scorer, cv=5)
grid_search.fit(X_train, y_train.toarray().ravel())

print("best params:", grid_search.best_params_)
print("best cross val score: {:.2f}".format(grid_search.best_score_))

C:\Users\12156\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\12156\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\12156\Desktop\homework\ml\project\AdaBoostWeak.py", line 95, in fit
    stump.fit(X, y, sample_weight = w_i)
  File "C:\Users\12156\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 903, in fit
    super().fit(
  File "C:\Users\12156\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 157, in fit
    X, y = self._validate_data(X, y,
  File "C:\Users\12156\anaconda3\lib\site-packages\sklearn\base.py", line 430, in _validate_data
    X = check_array(X, **check_X_params)
  File "C:\Users\12156\anaconda3\lib\site-packages\sklearn\utils\valid

MemoryError: Unable to allocate 1.81 MiB for an array with shape (236903,) and data type int64